In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
import sys

In [3]:
path = os.getcwd()

folders = []

for r, d, f in os.walk(path):
    for folder in d:
        if folder == ".ipynb_checkpoints":
            continue
        else:
            print(folder)
            folders.append(os.path.join(r, folder))

for f in folders:
    print(f)

Dial_Left_to_Right
Dial_Right_to_Left
Down_to_up
Left_to_right
Point
Right_to_Left
Up-to-Down
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Dial_Left_to_Right
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Dial_Right_to_Left
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Down_to_up
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Left_to_right
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Point
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Right_to_Left
C:\Users\OWNER\Documents\Projects\God\Smart-Home-Project\Machine Learning\Up-to-Down


In [4]:
ds_move = []
ds_var = [];
ds_std = [];
ds_slp = [];
ds_mean = []

for x in range(len(folders)):
   temp = os.listdir(folders[x])
   os.chdir(folders[x])
   print(x)
   #print("Temp is changing folders to ", temp)
   for c in range(len(os.listdir(folders[x]))):
       #if not c.strip(): continue  # skip the empty line
       #print(str(x) + 'aya');
       dataset = pd.read_csv(temp[c], sep=",", header=None);
       dataset.columns = ["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"];
       dataset = dataset.drop(["mag_x", "mag_y", "mag_z"], axis = 1)
       ds_move.append(x);
       ds_mean.append((dataset.mean()/10000).tolist());
       ds_var.append(dataset.var().tolist());
       ds_std.append(dataset.std().tolist());
       ds_slp.append(dataset.apply(lambda x: np.polyfit(dataset.index, x, 1)[0]).tolist());


0
1
2
3
4
5
6


In [5]:
test_1 = ds_mean[10]
test_2 = ds_mean[2000]
test_3 = ds_mean[1000]
tests = [test_1, test_2, test_3]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds_mean, ds_move, test_size = 0.25)

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
GAMMA = 0.5
C = 0.5
classifier = SVC(probability=True) #, gamma=GAMMA, C= C
parameters = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
              }
grid_search = GridSearchCV(estimator=classifier, param_grid=parameters, scoring="accuracy", cv=4)
grid_search = grid_search.fit(X_train, y_train)
df = pd.DataFrame(grid_search.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search.cv_results_)
print('\n\n')
print(grid_search.best_params_)
print(grid_search.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([0.05759054, 0.27025247, 0.20793283, 0.26226759, 0.05535913,
       0.13314551, 0.15233189, 0.22288501, 0.05583334, 0.07653838,
       0.13462752, 0.2553075 , 0.05534744, 0.05685747, 0.12540287,
       0.33297527, 0.05558765, 0.0498758 , 0.12091804, 0.38989818,
       0.05585945, 0.04585016, 0.11892623, 0.44663137, 0.05559701,
       0.0461275 , 0.1191653 , 0.50596356, 0.05596823, 0.04612803,
       0.11857295, 0.51607251, 0.0555883 , 0.04687035, 0.121279  ,
       0.53751099, 0.04811645, 0.23785442, 0.14535809, 0.19295257,
       0.04836529, 0.09597826, 0.10991544, 0.17252755, 0.04761803,
       0.06057763, 0.0984661 , 0.20019859, 0.04786724, 0.05061477,
       0.09423876, 0.26215702, 0.04785401, 0.04637152, 0.09149826,
       0.3138755 , 0.04810697, 0.04562318, 0.09025121, 0.36972064,
       0.04737896, 0.04685777, 0.09072274, 0.42108464, 0.047867  ,
       0.046

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

classifier_rf = RandomForestClassifier()

parameters_rf = {'n_estimators': [100, 115, 125, 135, 150],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'max_features': ['auto', 'sqrt', 'log2']
             }
grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=parameters_rf, scoring="accuracy", cv=5)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
df = pd.DataFrame(grid_search_rf.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search_rf.cv_results_)
print('\n\n')
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([0.20594716, 0.23313198, 0.25649252, 0.27388215, 0.30485702,
       0.20895643, 0.23995314, 0.25147309, 0.27198181, 0.30535674,
       0.20534391, 0.22950993, 0.25937662, 0.2770833 , 0.31355476,
       0.20148768, 0.22895627, 0.25056739, 0.27959876, 0.30290451,
       0.19914265, 0.22934861, 0.2484571 , 0.27707891, 0.30050864,
       0.20191426, 0.24059319, 0.24920254, 0.277811  , 0.29855595,
       0.20726404, 0.23680048, 0.25035543, 0.27595601, 0.29895306,
       0.20072026, 0.23235874, 0.25973544, 0.2708684 , 0.31519985,
       0.19808125, 0.22362289, 0.24339719, 0.2607172 , 0.29249272,
       0.19588146, 0.22983031, 0.24522982, 0.26684613, 0.2908577 ,
       0.19528356, 0.22580523, 0.24288568, 0.2614532 , 0.29872847,
       0.19848108, 0.22832108, 0.25167022, 0.26411939, 0.28949647,
       0.19344916, 0.21971269, 0.24230123, 0.27649326, 0.2912343 ,
       0.192

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

classifier_rf = KNeighborsClassifier()

parameters_rf = {'n_neighbors': [8,9,10,11,12,13],
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'leaf_size': [20, 30, 40, 50]
             }
grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=parameters_rf, scoring="accuracy", cv=4)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
df = pd.DataFrame(grid_search_rf.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search_rf.cv_results_)
print('\n\n')
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([0.00279361, 0.00298244, 0.00299257, 0.00299466, 0.0025211 ,
       0.00300026, 0.00323141, 0.00299197, 0.00250566, 0.00224251,
       0.00249368, 0.00247973, 0.00298446, 0.00275791, 0.0025261 ,
       0.00248307, 0.00273633, 0.00299478, 0.00273687, 0.00292045,
       0.00299001, 0.00275397, 0.00249815, 0.00249296, 0.00273967,
       0.00223893, 0.00250053, 0.00246531, 0.00296646, 0.00247645,
       0.00223738, 0.00298959, 0.00402415, 0.00224513, 0.00299358,
       0.00272655, 0.00274342, 0.0022788 , 0.00276285, 0.00295252,
       0.00226033, 0.00274444, 0.00273722, 0.00227702, 0.00298113,
       0.00223452, 0.0025112 , 0.00297588, 0.00249594, 0.00275695,
       0.00274438, 0.0024929 , 0.00324291, 0.0027445 , 0.00249368,
       0.0027436 , 0.00248075, 0.00259906, 0.00225234, 0.00199556,
       0.002482  , 0.00270927, 0.00273657, 0.00300711, 0.00273901,
       0.003

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

classifier_rf = GradientBoostingClassifier()

parameters_rf = {'n_estimators': [100, 115, 125, 135, 150],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'max_features': ['auto', 'sqrt', 'log2'],
              'learning_rate': [0.05,0.1,0.2],
              'subsample': [0.7,0.9,1]
             }
grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=parameters_rf, scoring="accuracy", cv=5)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
df = pd.DataFrame(grid_search_rf.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search_rf.cv_results_)
print('\n\n')
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([0.98439722, 1.01794758, 1.02145333, ..., 1.00789857, 0.9888751 ,
       0.95124364]), 'std_fit_time': array([0.0081856 , 0.00785364, 0.00931679, ..., 0.00825815, 0.00819453,
       0.00332165]), 'mean_score_time': array([0.00499864, 0.00496116, 0.00518942, ..., 0.00598168, 0.00558558,
       0.00599122]), 'std_score_time': array([2.45547525e-05, 5.16125509e-05, 3.97362941e-04, ...,
       3.35145634e-06, 4.83598636e-04, 1.43035252e-05]), 'param_learning_rate': masked_array(data=[0.05, 0.05, 0.05, ..., 0.2, 0.2, 0.2],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_features': masked_array(data=['auto', 'auto', 'auto', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_leaf'

In [ ]:
'''
Results of first KNeighbors

{'algorithm': 'auto', 'leaf_size': 20, 'n_neighbors': 10, 'weights': 'distance'}
0.9352112676056338

'''

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

classifier_rf = GradientBoostingClassifier()

parameters_rf = {'n_estimators': [200, 170, 165],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'max_features': ['auto', 'sqrt', 'log2'],
              'learning_rate': [0.05,0.1,0.2],
              'subsample': [0.7,0.9,1]
             }
grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=parameters_rf, scoring="accuracy", cv=5)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
df = pd.DataFrame(grid_search_rf.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search_rf.cv_results_)
print('\n\n')
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)

Empty DataFrame
Columns: [acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z, mag_x, mag_y, mag_z]
Index: []
{'mean_fit_time': array([1.87830553, 1.90402641, 1.87881703, ..., 1.10476956, 1.09986739,
       1.05459099]), 'std_fit_time': array([0.01319424, 0.01165016, 0.00873885, ..., 0.00787565, 0.0086233 ,
       0.00713394]), 'mean_score_time': array([0.00836005, 0.00797696, 0.00797057, ..., 0.00658088, 0.00618515,
       0.00618711]), 'std_score_time': array([4.67306366e-04, 3.10274095e-06, 8.46570626e-06, ...,
       4.88074066e-04, 3.98008684e-04, 4.02979046e-04]), 'param_learning_rate': masked_array(data=[0.05, 0.05, 0.05, ..., 0.2, 0.2, 0.2],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_max_features': masked_array(data=['auto', 'auto', 'auto', ..., 'log2', 'log2', 'log2'],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_leaf'

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

classifier_rf = GradientBoostingClassifier()

parameters_rf = {'n_estimators': [215, 225, 250],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4, 5],
              'max_features': ['auto', 'sqrt', 'log2'],
              'learning_rate': [0.05,0.1,0.2],
              'subsample': [0.7,0.9,1]
             }
grid_search_rf = GridSearchCV(estimator=classifier_rf, param_grid=parameters_rf, scoring="accuracy", cv=5)
grid_search_rf = grid_search_rf.fit(X_train, y_train)
df = pd.DataFrame(grid_search_rf.cv_results_, columns=["acc_x", "acc_y", "acc_z", "gyr_x", "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z"])
print(df)
print(grid_search_rf.cv_results_)
print('\n\n')
print(grid_search_rf.best_params_)
print(grid_search_rf.best_score_)